In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdfsss2/pdf2.pdf
/kaggle/input/pdfsss2/pdf4.pdf
/kaggle/input/pdfsss2/pdf1.pdf
/kaggle/input/pdfsss2/pdf3.pdf
/kaggle/input/pdfsss/pdf2.pdf
/kaggle/input/pdfsss/pdf1.pdf
/kaggle/input/pdfsss/pdf3.pdf
/kaggle/input/pdfsss3/pdf2.pdf
/kaggle/input/pdfsss3/pdf5.pdf
/kaggle/input/pdfsss3/pdf4.pdf
/kaggle/input/pdfsss3/pdf8.pdf
/kaggle/input/pdfsss3/pdf1.pdf
/kaggle/input/pdfsss3/pdf11.pdf
/kaggle/input/pdfsss3/pdf7.pdf
/kaggle/input/pdfsss3/pdf10.pdf
/kaggle/input/pdfsss3/pdf9.pdf
/kaggle/input/pdfsss3/pdf3.pdf
/kaggle/input/pdfsss3/pdf6.pdf


In [2]:
pip install pdfplumber pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pdfplumber
import pandas as pd
import os

# Function to extract all text from a PDF and save it to CSV
def convert_pdf_to_word_csv(pdf_path, output_dir):
    """
    Extracts all text from a PDF file, splits it by spaces into individual words,
    and saves it as a CSV file where each word is in a separate cell.
    
    Parameters:
    - pdf_path: Path to the PDF file
    - output_dir: Directory to save the output CSV file
    
    Returns:
    - output_csv: Path to the generated CSV file
    """
    output_csv = os.path.join(output_dir, os.path.basename(pdf_path).replace(".pdf", ".csv"))

    # Initialize an empty list to store rows (each row is a list of words)
    word_rows = []

    try:
        # Open the PDF file using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            # Iterate through all the pages in the PDF
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    # Split each line into words based on space and add to rows
                    lines = text.split('\n')
                    for line in lines:
                        words = line.split()  # Splitting the line by spaces into words
                        word_rows.append(words)

        # Create a DataFrame from the extracted words
        df = pd.DataFrame(word_rows)

        # Print the entire DataFrame to see its content before saving
        print(f"\nData extracted from {pdf_path}:\n", df)

        # Perform transformations if "CANOPY" is in the first row, first column
        if df.iloc[0, 0] == "CANOPY":
            # Get the DataFrame dimensions and print them
            pd_rows, pd_cols = df.shape
            print(f"\nDimensions of the DataFrame: Rows = {pd_rows}, Columns = {pd_cols}")

            # Insert a blank row at the end
            df.loc[pd_rows] = [''] * pd_cols

            # Insert a new row with column names after the blank row
            column_names = [
                'INSURANCE_PROVIDER', 'CHEQUE_#', 'CHEQUE_DATE', 'BATCH_ID', 'DATE_FILLED', 
                'CLAIM_#', 'MEMBER_ID', 'NAME', 'BENEFIT', 'REF._NO', 'STAT', 'CHARGED', 
                'COPAY', 'EXCLUDED', 'DEDUCTIBLE', '%', 'PAYABLE', 'FEE', 'GCT', 'TOTAL', 
                'REJECT_REASON', 'REMARK_CODE'
            ]
            df.loc[pd_rows + 1] = column_names

            # Copy data from [row 5 to pd_rows-5, column 0 to column 12] into new rows
            data_to_copy = df.iloc[5:pd_rows-5, 0:13]
            df = pd.concat([df, data_to_copy], ignore_index=True)

            print(f"\nDataFrame after transformations:\n", df)

        # Write the DataFrame (after transformations, if any) into a CSV file
        df.to_csv(output_csv, index=False, header=False, encoding='utf-8')
        print(f"\nCSV {output_csv} created successfully from {pdf_path}.\n")
    
    except Exception as e:
        print(f"Error while converting {pdf_path} to CSV: {e}")

    return output_csv

# Function to import CSV files into Excel
def import_csvs_to_excel(csv_files, excel_path):
    """
    Imports the word-separated CSV files into an Excel workbook, each CSV in a separate sheet.
    
    Parameters:
    - csv_files: List of paths to the CSV files
    - excel_path: Path to the Excel file
    """
    # Create an Excel writer object
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Loop through each CSV file
        for csv_file in csv_files:
            try:
                # Read the CSV file as a pandas DataFrame (words are already separated)
                df = pd.read_csv(csv_file, header=None)

                # Check if the DataFrame is empty
                if df.empty:
                    print(f"Warning: {csv_file} is empty or cannot be parsed.")
                else:
                    # Generate a sheet name based on the file name (without extension)
                    sheet_name = os.path.basename(csv_file).replace('.csv', '')

                    # Write the DataFrame to a new sheet in the Excel workbook
                    df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)
                    print(f"Data from {csv_file} has been imported into Excel sheet '{sheet_name}'.")
            except pd.errors.EmptyDataError:
                print(f"Error: {csv_file} is empty or has no data to parse.")
            except Exception as e:
                print(f"Error while importing {csv_file} to Excel: {e}")

    print(f"All CSV files have been imported into {excel_path}")

# Function to create a new Excel sheet with specific column names
def create_custom_sheet(excel_path):
    """
    Creates a new sheet in the Excel workbook with specified column names.
    
    Parameters:
    - excel_path: Path to the Excel workbook
    """
    column_names = [
        'INSURANCE_PROVIDER', 'CHEQUE_#', 'CHEQUE_DATE', 'BATCH_ID', 'DATE_FILLED', 
        'CLAIM_#', 'MEMBER_ID', 'NAME', 'BENEFIT', 'REF._NO', 'STAT', 'CHARGED', 
        'COPAY', 'EXCLUDED', 'DEDUCTIBLE', '%', 'PAYABLE', 'FEE', 'GCT', 'TOTAL', 
        'REJECT_REASON', 'REMARK_CODE'
    ]
    
    # Create a DataFrame with these column names and an empty row
    df = pd.DataFrame(columns=column_names)

    # Append the new sheet to the existing Excel workbook
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
        # Write the DataFrame to a new sheet in the Excel workbook
        df.to_excel(writer, sheet_name='Custom_Sheet', index=False)
        print(f"Custom sheet with specified columns has been added to {excel_path}")

# Main function to automate PDF to Excel conversion and add a custom sheet
def automate_pdf_to_excel(pdf_files, output_directory, excel_file):
    """
    Automates the process of converting PDFs to CSV (with words split into separate cells) 
    and importing them into Excel, along with adding a custom sheet with specific columns.
    
    Parameters:
    - pdf_files: List of paths to the PDF files
    - output_directory: Directory to save the converted CSVs
    - excel_file: Path to the final Excel file
    """
    # List to store paths of converted CSV files
    csv_files = []

    # Convert each PDF to CSV (splitting text into words) and store the CSV paths
    for pdf_file in pdf_files:
        csv_file = convert_pdf_to_word_csv(pdf_file, output_directory)
        csv_files.append(csv_file)

    # Import all CSV files into the Excel file
    import_csvs_to_excel(csv_files, excel_file)

    # Add a custom sheet with the specified columns
    create_custom_sheet(excel_file)

# Paths to the PDF files (replace with your actual file paths)
pdf_files = [
    "/kaggle/input/pdfsss3/pdf1.pdf",
    "/kaggle/input/pdfsss3/pdf2.pdf",
    "/kaggle/input/pdfsss3/pdf3.pdf",
    "/kaggle/input/pdfsss3/pdf4.pdf",
    "/kaggle/input/pdfsss3/pdf5.pdf",
    "/kaggle/input/pdfsss3/pdf6.pdf",
    "/kaggle/input/pdfsss3/pdf7.pdf",
    "/kaggle/input/pdfsss3/pdf8.pdf",
    "/kaggle/input/pdfsss3/pdf9.pdf",
    "/kaggle/input/pdfsss3/pdf10.pdf",
    "/kaggle/input/pdfsss3/pdf11.pdf"
]

# Directory where the converted CSVs will be saved
output_directory = "/kaggle/working/"

# Path to the final Excel file where the data will be stored
excel_file = "/kaggle/working/imported_data.xlsx"

# Run the automation process to extract text from PDFs and import to Excel
automate_pdf_to_excel(pdf_files, output_directory, excel_file)


Data extracted from /kaggle/input/pdfsss3/pdf1.pdf:
              0          1              2        3           4          5   \
0      Provider     Cheque        Details     None        None       None   
1        CANOPY  INSURANCE       LIMITED.     None        None       None   
2      Provider          :            PRO   PHYSIO           -   APH02315   
3         Batch         ID              :     9088        None       None   
4           Dt.     Filled         Claim#   Member          ID       Name   
5   08-AUG-2024    0007618  7217197599984  MELANIE      BITTER     PHYSIO   
6   08-AUG-2024    0007620  7207522899984   CAREEN       NOLAN     PHYSIO   
7   08-AUG-2024    0007621  7475204359984   CAREEN       NOLAN     PHYSIO   
8   08-AUG-2024    0007624  7334407179984     ANNA    MELVILLE     PHYSIO   
9   10-AUG-2024    0007645  7325586279984      IAN      GRAHAM     PHYSIO   
10  10-AUG-2024    0007647  7216486250298  RICARDO  LIVINGSTON     PHYSIO   
11    42,800.00  24,90